In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy.io
import os
import glob, itertools
import argparse, random

import torch
from torch import nn, optim
import torchvision
from torchvision import transforms, datasets, models, utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image, make_grid
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
from skimage import io, transform
from torch.optim import lr_scheduler
from skimage.transform import AffineTransform, warp
from tqdm import tqdm_notebook as tqdm

In [ ]:
## 파라미터 정의하기
# number of epochs of training
n_epochs = 20
# size of the batches
batch_size = 16
# adam: learning rate
lr = 0.00008
# adam: decay of first order momentum of gradient
b1 = 0.5
# adam: decay of second order momentum of gradient
b2 = 0.999

cuda = torch.cuda.is_available()

In [ ]:
## 데이터 가져오기
# Training data
file_train = h5py.File(f'/content/drive/MyDrive/산학협동강좌/AItopopt/Train_data.hdf5')
iters_train = file_train['iters'][:]
targets_train = file_train['targets'][:]
datanum = iters_train.shape[0]
print(f"Training Data: {datanum}")


#Test data
file_test = h5py.File(f'/content/drive/MyDrive/산학협동강좌/AItopopt/Test_data.hdf5')
iters_test = file_test['iters'][:]
targets_test = file_test['targets'][:]
datanum = targets_train.shape[0]
print(f"Testing Data: {datanum}")

Training Data: 7000
Testing Data: 7000


C:\Users\Femur\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:3: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Femur\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:11: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
## Dataset 만들기
class MyData(Dataset):
    def __init__(self, train, transform = None):
        if train == True:
            self.x = iters_train
            self.y = targets_train
        else:
            self.x = iters_test
            self.y = targets_test

        #Applying transformation
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        image_iters = np.moveaxis(self.x[idx], -1, 0).astype('float')
        image_targets = np.moveaxis(self.y[idx], -1, 0).astype('float')
        sample={'iters': image_iters, 'targets': image_targets}

        #Applying transformation
        if self.transform:
            sample=self.transform(sample)

        return sample

In [ ]:
## 텐서로 변환하기
class ToTensor(object):
    def __call__(self, sample):
        image_iters, image_targets = sample['iters'], sample['targets']

        image_iters=torch.from_numpy(image_iters)
        image_targets=torch.from_numpy(image_targets)

        return {'iters': image_iters, 'targets': image_targets}

In [ ]:
## DataLoader로 학습용 데이터 준비하기
transformed_train_data = MyData(train=True, transform=transforms.Compose([ToTensor()]))
transformed_test_data = MyData(train=False, transform=transforms.Compose([ToTensor()]))
train_dataloader = DataLoader(transformed_train_data, batch_size=16, shuffle=True, num_workers=0)
test_dataloader = DataLoader(transformed_test_data, batch_size=16, shuffle=True, num_workers=0)

In [ ]:
## 네트워크 구축하기
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        return x

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.encoder_conv1 = DoubleConv(in_channels, 64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder_conv2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder_conv3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder_conv4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder_conv5 = DoubleConv(512, 1024)

        self.upconv6 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder_conv6 = DoubleConv(1024, 512)
        self.upconv7 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder_conv7 = DoubleConv(512, 256)
        self.upconv8 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder_conv8 = DoubleConv(256, 128)
        self.upconv9 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder_conv9 = DoubleConv(128, 64)

        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.encoder_conv1(x)
        x2 = self.encoder_conv2(self.pool1(x1))
        x3 = self.encoder_conv3(self.pool2(x2))
        x4 = self.encoder_conv4(self.pool3(x3))
        x5 = self.encoder_conv5(self.pool4(x4))

        x = self.upconv6(x5)
        x = torch.cat([x, x4], dim=1)
        x = self.decoder_conv6(x)
        x = self.upconv7(x)
        x = torch.cat([x, x3], dim=1)
        x = self.decoder_conv7(x)
        x = self.upconv8(x)
        x = torch.cat([x, x2], dim=1)
        x = self.decoder_conv8(x)
        x = self.upconv9(x)
        x = torch.cat([x, x1], dim=1)
        x = self.decoder_conv9(x)

        x = self.final_conv(x)
        return x

In [ ]:
## 손실함수 및 옵티마이저
# Initialize generator and discriminator
generator = UNet(1,1)

# Loss function
fn_loss = torch.nn.L1Loss()

if cuda:
    generator = generator.cuda()
    fn_loss = fn_loss.cuda()

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

In [ ]:
## 네트워크 학습하기

train_gen_losses, train_disc_losses, train_counter = [], [], []
test_gen_losses, test_disc_losses = [], []
test_counter = [idx*len(train_dataloader.dataset) for idx in range(1, n_epochs+1)]

for epoch in range(n_epochs):

    ### Training
    gen_loss = 0
    tqdm_bar = tqdm(train_dataloader, desc=f'Training Epoch {epoch} ', total=int(len(train_dataloader)))
    for batch_idx, imgs in enumerate(tqdm_bar):
        generator.train()
        # Configure model input
        imgs_iters = Variable(imgs["iters"].type(Tensor))
        imgs_targets = Variable(imgs["targets"].type(Tensor))
        ### Train Generator
        optimizer_G.zero_grad()
        # Generate (near) optimal images from input
        gen_opt = generator(imgs_iters)
        # loss
        loss_G = fn_loss(gen_opt, imgs_targets)
        loss_G.backward()
        optimizer_G.step()

        gen_loss += loss_G.item()
        train_gen_losses.append(loss_G.item())
        train_counter.append(batch_idx*batch_size + imgs_iters.size(0) + epoch*len(train_dataloader.dataset))
        tqdm_bar.set_postfix(gen_loss=gen_loss/(batch_idx+1))

    # Testing
    gen_loss = 0
    tqdm_bar = tqdm(test_dataloader, desc=f'Testing Epoch {epoch} ', total=int(len(test_dataloader)))
    for batch_idx, imgs in enumerate(tqdm_bar):
        generator.eval()
        # Configure model input
        imgs_iters = Variable(imgs["iters"].type(Tensor))
        imgs_targets = Variable(imgs["targets"].type(Tensor))
        ### Eval Generator
        # Generate (near) optimal images from input
        gen_opt = generator(imgs_iters)
        # loss
        loss_G = fn_loss(gen_opt, imgs_targets)

        gen_loss += loss_G.item()
        tqdm_bar.set_postfix(gen_loss=gen_loss/(batch_idx+1))

    test_gen_losses.append(gen_loss/len(test_dataloader))

    # Save model checkpoints
    if np.argmin(test_gen_losses) == len(test_gen_losses)-1:
        torch.save(generator.state_dict(), "generator_opt.pth")

C:\Users\Femur\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED